In [53]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
import random

In [3]:
city = 'irvine'
state = 'ca'
url = f'https://www.compass.com/homes-for-sale/{(city)}-{(state)}/'
page = requests.get(url)
soup = bs(page.content, 'html.parser')

In [87]:
times = [4,5,6,7,]
police_df = pd.read_csv('Data/police_df.csv')
soup_bowl = []
state = 'CA'
for city in police_df['City']:
    city_clean = city.replace(' ','-')
    url = f'https://www.compass.com/homes-for-sale/{city_clean.lower()}-{state.lower()}/'
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    soup_bowl.append(soup)
    time.sleep(random.choice(times))
len(soup_bowl)

34

In [88]:
price_list = []
for soup in soup_bowl:
    prices = soup.find_all('div', class_='uc-listingCard-mainStats')
    for price in prices:
        clean_price = price.text.replace('$','')
        clean_price = clean_price.replace(',','')
        price_list.append(int(clean_price))

In [90]:
#They placed a div tag inside a div tag for some unknown reason. Calling the 'text' method
#does not separate the number of beds from the word 'beds' with a space. This is my work around.

bed_list = []
bath_list = []
sqft_list = []
for soup in soup_bowl:
    beds = soup.find_all('div', class_='uc-listingCard-subStat uc-listingCard-subStat--beds')
    baths = soup.find_all('div', class_='uc-listingCard-subStat uc-listingCard-subStat--baths')
    sqfts = soup.find_all('div', class_='uc-listingCard-subStat uc-listingCard-subStat--sqFt')

    for i in range(len(beds)):
        bed_list.append(str(beds[i]).split('<')[1].split('>')[1])
        bath_list.append(str(baths[i]).split('<')[1].split('>')[1])
        sqft = str(sqfts[i]).split('<')[1].split('>')[1]
        if ',' in sqft:
            sqft_list.append(float(sqft.replace(',', '')))
        else:
            sqft_list.append(str(sqft))

In [104]:
cities = []
streets = []
zipcodes = []
for i,soup in enumerate(soup_bowl):
    addresses = soup.find_all('h2', class_='uc-listingCard-title truncate-overflow-text')
    for address in addresses:
        city = police_df['City'][i]
        street = str(address).split('target="_blank">')[1].split('</a>')[0]
        zipcode = str(address).split(', CA ')[1].split('</h2>')[0]
        cities.append(city)
        streets.append(street)
        zipcodes.append(zipcode)


In [105]:
Listings = pd.DataFrame({
    'Street':streets,    
    'City':cities,
    'Zipcode':zipcodes,
    'Listing Price ($)':price_list,
    'Number of Beds':bed_list,
    'Number of Baths':bath_list,
    'Square Footage': sqft_list
})

Listings['House ID'] = [i for i in range(len(Listings))]
Listings = Listings.set_index('House ID')

In [197]:
Clean_Listings = pd.DataFrame([])
for i in range(len(Listings)):
    try:
        float(Listings[Listings.index == i]['Number of Baths'][i])
        float(Listings[Listings.index == i]['Number of Beds'][i])
        float(Listings[Listings.index == i]['Listing Price ($)'][i])
        Clean_Listings = Clean_Listings.append(Listings[Listings.index == i])
    except:
        continue
        
Clean_Listings['House ID'] = [i for i in range(len(Clean_Listings))]
Clean_Listings = Clean_Listings.set_index('House ID')

In [199]:
Clean_Listings.to_csv('Data/Home_Listings.csv')